In [5]:
import pandas as pd
import numpy as np
import re
import os

In [40]:
path = "krx_4월_첫거래일"
ju_file_list = os.listdir(path)
csv_df_li = []
for f in ju_file_list:
    df = pd.read_csv(f"./krx_4월_첫거래일/{f}", encoding='euc-kr')
    year = f.replace('.csv', '')
    df['년도'] = year
    csv_df_li.append(df)

df = pd.concat(csv_df_li)
df = df.sort_values(['종목명', '년도'], ascending=False)
companys = df['종목명'].unique()
fi_df_dic = {}
for co in companys:
    fi_df = df[df['종목명'] == co][['종가','시가총액', '상장주식수']].T
    fi_df.columns = df[df['종목명'] == co]['년도'].values
    fi_df_dic[co] = fi_df

In [41]:
path = "./fsdata"
file_list = os.listdir(path)

In [12]:
def concat_df(df, name, name_pos):
    # 칼럼 이름 정리, name칼럼 값 통합
    col_list = []
    for i, col in enumerate(df.columns):
        try:
            int(col[:8])
            col_list.append(i)
        except:
            pass

    nm_df = df.iloc[:, name_pos]
    ye_df =  df.iloc[:, col_list]
    con_df = pd.concat([nm_df, ye_df], axis=1)
    filter_col = ['name']
    for col in con_df.columns[1:]:
        filter_col.append(col[:4])
    con_df.columns = filter_col
    con_df.set_index('name', inplace=True)
    con_df.index = [name]
    
   
    #  칼럼 인덱싱 중복 제거
    con_df = con_df.T.reset_index().drop_duplicates(subset=['index']).set_index('index').T
    con_df.columns.name = ''
    if '2009' in con_df.columns:
        con_df = con_df.drop('2009', axis=1)
    if '2021' in con_df.columns:
        con_df = con_df.drop('2021', axis=1)
        
    return con_df

In [27]:
def df1_Extraction(df, label_col, sheet_name):
    df1 = df['Data_bs'][df['Data_bs'].iloc[:, label_col].str.contains('자본금').fillna(False)] 
    if len(df1) > 1:
        df1 = df['Data_bs'][df['Data_bs'].iloc[:, label_col] == '자본금']
    elif len(df1) == 0:
        df1 = df['Data_bs'][df['Data_bs'].iloc[:, 1].str.contains('ifrs-full_IssuedCapital').fillna(False)] 
        if len(df1) == 0:
            df1 = df['Data_bs'][df['Data_bs'].iloc[:, 1].isin(['ifrs_EquityAttributableToOwnersOfParent']).fillna(False)] 
    if len(df1) == 0:
        df1 = df['Data_bs'][df['Data_bs'].iloc[:, label_col].isin(['납입자본'])] 
    return df1
        
def df2_Extraction(df, label_col, sheet_name):
    df2 = df['Data_bs'][df['Data_bs'].iloc[:, label_col] == '자산총계']
    return df2

def df3_Extraction(df, label_col, sheet_name):
    df3 = df['Data_cis'][df['Data_cis'].iloc[:, label_col].str.contains('당기순이익').fillna(False)]
    if len(df3) > 1:
        df3 = df['Data_cis'][df['Data_cis'].iloc[:, label_col].isin(['연결당기순이익', '당기순이익(손실)', '당기순이익', '당기순손익', '당기순손실'])]
        if len(df3) == 0:
            df3 = df['Data_cf'][df['Data_cf'].iloc[:, label_col].isin(['당기순이익(손실)'])]
    elif len(df3) == 0:
        df3 = df['Data_cis'][df['Data_cis'].iloc[:, label_col].isin(['당기순손익', '당기순손실', '당기의 순이익', '계속영업이익(손실)'])]
        if len(df3) == 0:
            df3 = df[sheet_name][df[sheet_name].iloc[:, 1].str.contains('당기순이익').fillna(False)]
        if len(df3) == 0:
            df3 = df['Data_cis'][df['Data_cis'].iloc[:, label_col].str.contains('당기손익').fillna(False)]
    if len(df3) > 1:
        df3 = df3[df3.isna().sum(axis = 1) == df3.isna().sum(axis = 1).min()]
    return df3

def df4_Extraction(df, label_col, sheet_name):
    regex = r'[1-9]?[A-z]?[\{\}\[\]\/?.,;:|\)*~`!^\-_+<>@\#$%&\\\=\(\'\"]?\s?영업\s?활동?([가-힣]+)?(\s+[가-힣]+\s+)?\s?현금\s?흐름'
    df4 = df['Data_cf'][df['Data_cf'].iloc[:, 1].str.contains('ifrs-full_CashFlowsFromUsedInOperatingActivities').fillna(False)]                          

    if len(df4) == 0:
        df4 = df['Data_cf'][df['Data_cf'].iloc[:, label_col].str.contains(regex).fillna(False)]
        if len(df4) > 1:
            df4 = df['Data_cf'][df['Data_cf'].iloc[:, 1].isin(['ifrs_CashFlowsFromUsedInOperatingActivities']).fillna(False)]
            if len(df4) > 0:
                df4 = df['Data_cf'][df['Data_cf'].iloc[:, 1].str.contains(r'영업\s활동\s현금\s흐름').fillna(False)]
    return df4

def df5_Extraction(df, label_col, sheet_name):
    df5 = df[sheet_name][df[sheet_name].iloc[:, 1].str.contains('dart_OperatingIncomeLoss').fillna(False)]
    if len(df5) == 0:
        df5 = df[sheet_name][df[sheet_name].iloc[:, label_col].str.contains(r'영업\s?이익').fillna(False)]
        if len(df5) == 0:
            df5 = df[sheet_name][df[sheet_name].iloc[:, 1].str.contains(r'영업\s?이익').fillna(False)]
            if len(df5) == 0:
                sheet_name = 'Data_cis'
                df5 = df[sheet_name][df[sheet_name].iloc[:, label_col].str.contains(r'영업\s?이익').fillna(False)]
    return df5

def df6_Extraction(df, label_col, sheet_name):
    df6 = df[sheet_name][df[sheet_name].iloc[:, 1].str.contains('ifrs-full_Revenue').fillna(False)] 
    if len(df6) == 0: # ID로 구하기
        df6 = df[sheet_name][df[sheet_name].iloc[:, 1].isin(['ifrs-full_Revenue', 'ifrs_Revenue'])]
    if len(df6) == 0: # Name으로 구하기
        df6 = df[sheet_name][df[sheet_name].iloc[:, label_col].str.contains('매출액').fillna(False)]
        if len(df6) == 0:
            df6 = df[sheet_name][df[sheet_name].iloc[:, label_col].isin(['매출'])]

        if len(df6) > 1: 
            df6 = df[sheet_name][df[sheet_name].iloc[:, label_col].str.contains(r'Ⅰ.매출액.*').fillna(False)]


    if len(df6) == 0: # name col다른 애들 col변경해서 잡기

        df6 = df[sheet_name][df[sheet_name].iloc[:, 1].str.contains('매출액').fillna(False)]
        if len(df6) > 1:
            df6 = df[sheet_name][df[sheet_name].iloc[:, 1].isin(['매출액'])]
        if len(df6) == 0:
            df6 = df[sheet_name][df[sheet_name].iloc[:, 1].isin(['I.매출'])]
        if len(df6) == 0:
            df6 = df['Data_cis'][df['Data_cis'].iloc[:, label_col].str.contains('매출액').fillna(False)]

    if len(df6) == 0:   # 매출액 없는녀석 영업수익 구하기
        df6 = df[sheet_name][df[sheet_name].iloc[:, label_col].str.contains('영업수익').fillna(False)]
        if len(df6) > 1: # name으로 isin
            df6 = df[sheet_name][df[sheet_name].iloc[:, 1].isin(['I.영업수익', '영업수익'])]
            if len(df6) > 1:
                df6 = df[sheet_name][df[sheet_name].iloc[:, 1].isin(['I.영업수익', '영업수익'])]
    elif len(df6) > 1:
        df6 = df[sheet_name][df[sheet_name].iloc[:, 1].isin(['ifrs-full_Revenue', 'ifrs_Revenue'])]
    return df6

In [34]:
def sheet_and_label_set(df):
    if df['Data_cis'].iloc[:, 1][0] == "label_ko":
            label_col = 1
    else:
        label_col = 2

    if 'Data_is' in df.keys():
        sheet_name = 'Data_is'
    else:
        sheet_name = 'Data_cis'
    return label_col, sheet_name

In [39]:
df_li = []
error_li = []
count = 0
for i, f in enumerate(file_list):
   
    df = pd.read_excel(f"./fsdata/{f}", sheet_name=None)
    label_col, sheet_name = sheet_and_label_set(df)
    
    df1 = df1_Extraction(df, label_col, sheet_name)
    df2 = df2_Extraction(df, label_col, sheet_name)
    df3 = df3_Extraction(df, label_col, sheet_name)
    df4 = df4_Extraction(df, label_col, sheet_name)
    df5 = df5_Extraction(df, label_col, sheet_name)
    df6 = df6_Extraction(df, label_col, sheet_name)
           
    df1 = concat_df(df1, '자본금', label_col)
    df2 = concat_df(df2, '자산총계', label_col)
    df3 = concat_df(df3, '당기순이익', label_col)
    df4 = concat_df(df4, '영업활동현금흐름', label_col)
    df5 = concat_df(df5, '영업이익', label_col)
    df6 = concat_df(df6, '매출액', label_col)
    file_n = f.replace('.xlsx', '')
    if file_n in fi_df_dic.keys():
        fi_df = fi_df_dic[file_n]
        con_df = pd.concat([df1, df2, df3, df4, df5, df6, fi_df])
        print(f, 'Concat Succeeded')
        con_df.to_csv(f'./data/{file_n}.csv')
        
        
        

print(error_li)
print(count)

3S.xlsx Concat Succeeded
AJ네트웍스.xlsx Concat Succeeded
AK홀딩스.xlsx Concat Succeeded
APS홀딩스.xlsx Concat Succeeded
AP시스템.xlsx Concat Succeeded
BGF.xlsx Concat Succeeded
BGF리테일.xlsx Concat Succeeded
BNK금융지주.xlsx Concat Succeeded
BYC.xlsx Concat Succeeded
CJ CGV.xlsx Concat Succeeded
CJ ENM.xlsx Concat Succeeded
CJ.xlsx Concat Succeeded
CJ대한통운.xlsx Concat Succeeded
CJ씨푸드.xlsx Concat Succeeded
CJ제일제당.xlsx Concat Succeeded
CJ프레시웨이.xlsx Concat Succeeded
CMG제약.xlsx Concat Succeeded
CS홀딩스.xlsx Concat Succeeded
DB.xlsx Concat Succeeded
DB하이텍.xlsx Concat Succeeded
DMS.xlsx Concat Succeeded
DRB동일.xlsx Concat Succeeded
DSR.xlsx Concat Succeeded
E1.xlsx Concat Succeeded
EDGC.xlsx Concat Succeeded
EG.xlsx Concat Succeeded
EMW.xlsx Concat Succeeded
ESR켄달스퀘어리츠.xlsx Concat Succeeded
GH신소재.xlsx Concat Succeeded
GS.xlsx Concat Succeeded
GS건설.xlsx Concat Succeeded
GS글로벌.xlsx Concat Succeeded
GS리테일.xlsx Concat Succeeded
HB테크놀러지.xlsx Concat Succeeded
HDC.xlsx Concat Succeeded
HDC현대EP.xlsx Concat Succeeded
HDC현

C:\Users\bitcamp\anaconda3\lib\site-packages\pandas\core\strings\accessor.py:101: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


 Concat Succeeded
SKC.xlsx Concat Succeeded
SK가스.xlsx Concat Succeeded
SK네트웍스.xlsx Concat Succeeded
SK디스커버리.xlsx Concat Succeeded
SK디앤디.xlsx Concat Succeeded
SK렌터카.xlsx Concat Succeeded
SK이노베이션.xlsx Concat Succeeded
SK케미칼.xlsx Concat Succeeded
SK텔레콤.xlsx Concat Succeeded
SK하이닉스.xlsx Concat Succeeded
SM C&C.xlsx Concat Succeeded
SM Life Design.xlsx Concat Succeeded
SNK.xlsx Concat Succeeded
SPC삼립.xlsx Concat Succeeded
STX.xlsx Concat Succeeded
STX엔진.xlsx Concat Succeeded
STX중공업.xlsx Concat Succeeded
TCC스틸.xlsx Concat Succeeded
THE E&M.xlsx Concat Succeeded
THE MIDONG.xlsx Concat Succeeded
TJ미디어.xlsx Concat Succeeded
TPC.xlsx Concat Succeeded
TS트릴리온.xlsx Concat Succeeded
UCI.xlsx Concat Succeeded
WI.xlsx Concat Succeeded
WISCOM.xlsx Concat Succeeded
YBM넷.xlsx Concat Succeeded
YG PLUS.xlsx Concat Succeeded
YTN.xlsx Concat Succeeded
YW.xlsx Concat Succeeded
가비아.xlsx Concat Succeeded
가온미디어.xlsx Concat Succeeded
가온전선.xlsx Concat Succeeded
강남제비스코.xlsx Concat Succeeded
강스템바이오텍.xlsx Concat Succ

C:\Users\bitcamp\anaconda3\lib\site-packages\pandas\core\strings\accessor.py:101: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


더네이쳐홀딩스.xlsx Concat Succeeded
더블유게임즈.xlsx Concat Succeeded
더존비즈온.xlsx Concat Succeeded
덕산테코피아.xlsx Concat Succeeded
덕산하이메탈.xlsx Concat Succeeded
덕신하우징.xlsx Concat Succeeded
덕양산업.xlsx Concat Succeeded
덕우전자.xlsx Concat Succeeded
데브시스터즈.xlsx Concat Succeeded
덱스터.xlsx Concat Succeeded
덴티움.xlsx Concat Succeeded
도화엔지니어링.xlsx Concat Succeeded
동구바이오제약.xlsx Concat Succeeded
동국S&C.xlsx Concat Succeeded
동국산업.xlsx Concat Succeeded
동국알앤에스.xlsx Concat Succeeded
동국제강.xlsx Concat Succeeded
동국제약.xlsx Concat Succeeded
동방선기.xlsx Concat Succeeded
동방아그로.xlsx Concat Succeeded
동부건설.xlsx Concat Succeeded
동성제약.xlsx Concat Succeeded
동성화인텍.xlsx Concat Succeeded
동아쏘시오홀딩스.xlsx Concat Succeeded
동아에스티.xlsx Concat Succeeded
동아엘텍.xlsx Concat Succeeded
동아지질.xlsx Concat Succeeded
동아타이어.xlsx Concat Succeeded
동아화성.xlsx Concat Succeeded
동양고속.xlsx Concat Succeeded
동양에스텍.xlsx Concat Succeeded
동양이엔피.xlsx Concat Succeeded
동양철관.xlsx Concat Succeeded
동양피스톤.xlsx Concat Succeeded
동우팜투테이블.xlsx Concat Succeeded
동원F&B.xlsx Concat Suc

...

C:\Users\bitcamp\anaconda3\lib\site-packages\pandas\core\strings\accessor.py:101: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


오성첨단소재.xlsx Concat Succeeded
오션브릿지.xlsx Concat Succeeded
오스코텍.xlsx Concat Succeeded
오스템.xlsx Concat Succeeded
오스템임플란트.xlsx Concat Succeeded
오이솔루션.xlsx Concat Succeeded
오킨스전자.xlsx Concat Succeeded
오텍.xlsx Concat Succeeded
오픈베이스.xlsx Concat Succeeded
올리패스.xlsx Concat Succeeded
올릭스.xlsx Concat Succeeded
옴니시스템.xlsx Concat Succeeded
옵트론텍.xlsx Concat Succeeded
와이솔.xlsx Concat Succeeded
와이아이케이.xlsx Concat Succeeded
와이엔텍.xlsx Concat Succeeded
와이엠씨.xlsx Concat Succeeded
와이엠티.xlsx Concat Succeeded
와이오엠.xlsx Concat Succeeded
와이제이엠게임즈.xlsx Concat Succeeded
와이지-원.xlsx Concat Succeeded
와이지엔터테인먼트.xlsx Concat Succeeded
와이팜.xlsx Concat Succeeded
용평리조트.xlsx Concat Succeeded
우리금융지주.xlsx Concat Succeeded
우리기술.xlsx Concat Succeeded
우리기술투자.xlsx Concat Succeeded
우리들휴브레인.xlsx Concat Succeeded
우리로.xlsx Concat Succeeded
우리바이오.xlsx Concat Succeeded
우리산업홀딩스.xlsx Concat Succeeded
우리손에프앤지.xlsx Concat Succeeded
우리이앤엘.xlsx Concat Succeeded
우리조명.xlsx Concat Succeeded
우수AMS.xlsx Concat Succeeded
우신시스템.xlsx Concat Succe

C:\Users\bitcamp\anaconda3\lib\site-packages\pandas\core\strings\accessor.py:101: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


이스트소프트.xlsx Concat Succeeded
이스트아시아홀딩스.xlsx Concat Succeeded
이아이디.xlsx Concat Succeeded
이엔에프테크놀로지.xlsx Concat Succeeded
이엔플러스.xlsx Concat Succeeded
이엘피.xlsx Concat Succeeded
이엠넷.xlsx Concat Succeeded
이엠앤아이.xlsx Concat Succeeded
이엠코리아.xlsx Concat Succeeded
이엠텍.xlsx Concat Succeeded
이연제약.xlsx Concat Succeeded
이오테크닉스.xlsx Concat Succeeded
이오플로우.xlsx Concat Succeeded
이원컴포텍.xlsx Concat Succeeded
이월드.xlsx Concat Succeeded
이즈미디어.xlsx Concat Succeeded
이지케어텍.xlsx Concat Succeeded
이지홀딩스.xlsx Concat Succeeded
이큐셀.xlsx Concat Succeeded
이크레더블.xlsx Concat Succeeded
이트론.xlsx Concat Succeeded
이화전기.xlsx Concat Succeeded
인디에프.xlsx Concat Succeeded
인바디.xlsx Concat Succeeded
인베니아.xlsx Concat Succeeded
인선이엔티.xlsx Concat Succeeded
인성정보.xlsx Concat Succeeded
인스코비.xlsx Concat Succeeded
인지디스플레.xlsx Concat Succeeded
인지컨트롤스.xlsx Concat Succeeded
인천도시가스.xlsx Concat Succeeded
인콘.xlsx Concat Succeeded
인크로스.xlsx Concat Succeeded
인탑스.xlsx Concat Succeeded
인터로조.xlsx Concat Succeeded
인터엠.xlsx Concat Succeeded
인터지스.xlsx

폴루스바이오팜.xlsx Concat Succeeded
푸드나무.xlsx Concat Succeeded
푸드웰.xlsx Concat Succeeded
푸른기술.xlsx Concat Succeeded
풀무원.xlsx Concat Succeeded
풍강.xlsx Concat Succeeded
풍국주정.xlsx Concat Succeeded
풍산.xlsx Concat Succeeded
퓨쳐켐.xlsx Concat Succeeded
프럼파스트.xlsx Concat Succeeded
프로텍.xlsx Concat Succeeded
프리시젼바이오.xlsx Concat Succeeded
프리엠스.xlsx Concat Succeeded
플랜티넷.xlsx Concat Succeeded
플레이위드.xlsx Concat Succeeded
플리토.xlsx Concat Succeeded
피노텍.xlsx Concat Succeeded
피에스엠씨.xlsx Concat Succeeded
피에스케이.xlsx Concat Succeeded
픽셀플러스.xlsx Concat Succeeded
필옵틱스.xlsx Concat Succeeded
핌스.xlsx Concat Succeeded
하나마이크론.xlsx Concat Succeeded
하나투어.xlsx Concat Succeeded
하림.xlsx Concat Succeeded
하림지주.xlsx Concat Succeeded
하이골드3호.xlsx Concat Succeeded
하이로닉.xlsx Concat Succeeded
하이록코리아.xlsx Concat Succeeded
하이소닉.xlsx Concat Succeeded
하이스틸.xlsx Concat Succeeded
하이즈항공.xlsx Concat Succeeded
하이트진로.xlsx Concat Succeeded
하이트진로홀딩스.xlsx Concat Succeeded
한국가구.xlsx Concat Succeeded
한국가스공사.xlsx Concat Succeeded
한국경제TV.xlsx Conca

In [588]:
df_li = []
error_li = []
count = 0
for i, f in enumerate(file_list):
   
        df = pd.read_excel(f"./fsdata/{f}", sheet_name=None)

        if df['Data_cis'].iloc[:, 1][0] == "label_ko":
            label_col = 1
        else:
            label_col = 2

        if 'Data_is' in df.keys():
            sheet_name = 'Data_is'
        else:
            sheet_name = 'Data_cis'

        # 자본금   
        df1 = df['Data_bs'][df['Data_bs'].iloc[:, label_col].str.contains('자본금').fillna(False)] 
        if len(df1) > 1:
            df1 = df['Data_bs'][df['Data_bs'].iloc[:, label_col] == '자본금']
        elif len(df1) == 0:
            df1 = df['Data_bs'][df['Data_bs'].iloc[:, 1].str.contains('ifrs-full_IssuedCapital').fillna(False)] 
            if len(df1) == 0:
                df1 = df['Data_bs'][df['Data_bs'].iloc[:, 1].isin(['ifrs_EquityAttributableToOwnersOfParent']).fillna(False)] 
        if len(df1) == 0:
            df1 = df['Data_bs'][df['Data_bs'].iloc[:, label_col].isin(['납입자본'])] 

        # 자산총계
        df2 = df['Data_bs'][df['Data_bs'].iloc[:, label_col] == '자산총계']

        # 당기순이익
        df3 = df['Data_cis'][df['Data_cis'].iloc[:, label_col].str.contains('당기순이익').fillna(False)]
        if len(df3) > 1:
            df3 = df['Data_cis'][df['Data_cis'].iloc[:, label_col].isin(['연결당기순이익', '당기순이익(손실)', '당기순이익', '당기순손익', '당기순손실'])]
            if len(df3) == 0:
                df3 = df['Data_cf'][df['Data_cf'].iloc[:, label_col].isin(['당기순이익(손실)'])]
        elif len(df3) == 0:
            df3 = df['Data_cis'][df['Data_cis'].iloc[:, label_col].isin(['당기순손익', '당기순손실', '당기의 순이익', '계속영업이익(손실)'])]
            if len(df3) == 0:
                df3 = df[sheet_name][df[sheet_name].iloc[:, 1].str.contains('당기순이익').fillna(False)]
            if len(df3) == 0:
                df3 = df['Data_cis'][df['Data_cis'].iloc[:, label_col].str.contains('당기손익').fillna(False)]
        if len(df3) > 1:
            df3 = df3[df3.isna().sum(axis = 1) == df3.isna().sum(axis = 1).min()]

        # 영업활동 현금흐름
        regex = r'[1-9]?[A-z]?[\{\}\[\]\/?.,;:|\)*~`!^\-_+<>@\#$%&\\\=\(\'\"]?\s?영업\s?활동?([가-힣]+)?(\s+[가-힣]+\s+)?\s?현금\s?흐름'
        df4 = df['Data_cf'][df['Data_cf'].iloc[:, 1].str.contains('ifrs-full_CashFlowsFromUsedInOperatingActivities').fillna(False)]                          

        if len(df4) == 0:
            df4 = df['Data_cf'][df['Data_cf'].iloc[:, label_col].str.contains(regex).fillna(False)]
            if len(df4) > 1:
                df4 = df['Data_cf'][df['Data_cf'].iloc[:, 1].isin(['ifrs_CashFlowsFromUsedInOperatingActivities']).fillna(False)]
                if len(df4) > 0:
                    df4 = df['Data_cf'][df['Data_cf'].iloc[:, 1].str.contains(r'영업\s활동\s현금\s흐름').fillna(False)]

        # 영업이익
        df5 = df[sheet_name][df[sheet_name].iloc[:, 1].str.contains('dart_OperatingIncomeLoss').fillna(False)]
        if len(df5) == 0:
            df5 = df[sheet_name][df[sheet_name].iloc[:, label_col].str.contains(r'영업\s?이익').fillna(False)]
            if len(df5) == 0:
                df5 = df[sheet_name][df[sheet_name].iloc[:, 1].str.contains(r'영업\s?이익').fillna(False)]
                if len(df5) == 0:
                    sheet_name = 'Data_cis'
                    df5 = df[sheet_name][df[sheet_name].iloc[:, label_col].str.contains(r'영업\s?이익').fillna(False)]

        # 매출액
        df6 = df[sheet_name][df[sheet_name].iloc[:, 1].str.contains('ifrs-full_Revenue').fillna(False)] 
        if len(df6) == 0: # ID로 구하기
            df6 = df[sheet_name][df[sheet_name].iloc[:, 1].isin(['ifrs-full_Revenue', 'ifrs_Revenue'])]
        if len(df6) == 0: # Name으로 구하기
            df6 = df[sheet_name][df[sheet_name].iloc[:, label_col].str.contains('매출액').fillna(False)]
            if len(df6) == 0:
                df6 = df[sheet_name][df[sheet_name].iloc[:, label_col].isin(['매출'])]

            if len(df6) > 1: 
                df6 = df[sheet_name][df[sheet_name].iloc[:, label_col].str.contains(r'Ⅰ.매출액.*').fillna(False)]


        if len(df6) == 0: # name col다른 애들 col변경해서 잡기

            df6 = df[sheet_name][df[sheet_name].iloc[:, 1].str.contains('매출액').fillna(False)]
            if len(df6) > 1:
                df6 = df[sheet_name][df[sheet_name].iloc[:, 1].isin(['매출액'])]
            if len(df6) == 0:
                df6 = df[sheet_name][df[sheet_name].iloc[:, 1].isin(['I.매출'])]
            if len(df6) == 0:
                df6 = df['Data_cis'][df['Data_cis'].iloc[:, label_col].str.contains('매출액').fillna(False)]

        if len(df6) == 0:   # 매출액 없는녀석 영업수익 구하기
            df6 = df[sheet_name][df[sheet_name].iloc[:, label_col].str.contains('영업수익').fillna(False)]
            if len(df6) > 1: # name으로 isin
                df6 = df[sheet_name][df[sheet_name].iloc[:, 1].isin(['I.영업수익', '영업수익'])]
                if len(df6) > 1:
                    df6 = df[sheet_name][df[sheet_name].iloc[:, 1].isin(['I.영업수익', '영업수익'])]
        elif len(df6) > 1:
            df6 = df[sheet_name][df[sheet_name].iloc[:, 1].isin(['ifrs-full_Revenue', 'ifrs_Revenue'])]
        
        df1 = concat_df(df1, '자본금', label_col)
        df2 = concat_df(df2, '자산총계', label_col)
        df3 = concat_df(df3, '당기순이익', label_col)
        df4 = concat_df(df4, '영업활동현금흐름', label_col)
        df5 = concat_df(df5, '영업이익', label_col)
        df6 = concat_df(df6, '매출액', label_col)
        file_n = f.replace('.xlsx', '')
        if file_n in fi_df_dic.keys():
            fi_df = fi_df_dic[file_n]
            con_df = pd.concat([df1, df2, df3, df4, df5, df6, fi_df])
            con_df.to_csv(f'./data/{file_n}.csv')
        
        
        #print(df1.index.values, df2.index.values, df3.index.values, df4.index.values, df5.index.values, df6.index.values, i, f)
       # print(i, f, len(df1), len(df2), len(df3), len(df4), len(df5), len(df6))
#         if len(df1) != 1 or len(df2) != 1 or len(df3) != 1 or len(df4) != 1 or len(df5) != 1 or len(df6) != 1:
#             count += 1
#             error_li.append([i, f])
#             print('Error')
    
print(error_li)
print(count)

C:\Users\bitcamp\anaconda3\lib\site-packages\pandas\core\strings\accessor.py:101: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)
C:\Users\bitcamp\anaconda3\lib\site-packages\pandas\core\strings\accessor.py:101: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)
C:\Users\bitcamp\anaconda3\lib\site-packages\pandas\core\strings\accessor.py:101: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)
C:\Users\bitcamp\anaconda3\lib\site-packages\pandas\core\strings\accessor.py:101: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)
C:\Users\bitcamp\anaconda3\lib\site-packages\pandas\core\strings\accessor.py:101: UserWarning: This pattern has match groups. To actually get the groups, use str.extrac

[]
0
